<a target="_blank" href="https://colab.research.google.com/github/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This tutorial will show how to use `mhar` for sampling full dimensional polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs. If you want to se a tutorial on how to sample non-full dimensional polytopes see [tutorial](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb)

In [1]:
import torch

Before starting let's check if you have an avaialble gpu device or not.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [3]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$

For fully dimensional polytopes we need to use the class `Polytope` in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-hypercube = \{x \in R^n || x \in [-1,1]^n \} $  

Which we can represent in matrix restrictions:  
$ Ix \leq 1$  
$ -Ix \leq 1$  
Where $I$ is the identity matrix of dimension $n \times n$ 


We will use this restrictions to define the polytope as:  
$A^Ix = [I | -I]x \leq 1 = b^I$

#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope.

In [4]:
n = 3 # Dimension
dtype = torch.float32 # Precision 
A_I = torch.cat((torch.eye(n), torch.eye(n) * -1.0), dim=0).to(dtype) # Inequality Matrix
b_I = torch.ones(2 * n, dtype=dtype).view(-1, 1)  # Inequality restriction vector      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}')

Inequality Matrix A^I 
 tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]]) 

Inequality Vector b^I 
 tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


Now lets create a `Polytope` object to represent the polytope.

In [5]:
from mhar.polytope import Polytope
hypercube = Polytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [6]:
hypercube

Numeric Precision (dtype) torch.float32
Device: cuda
A_in: torch.Size([6, 3]) 
b_in: torch.Size([6, 1])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [7]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [8]:
x0 = ChebyshevCenter(polytope=hypercube, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device='cuda', # device used cpu or cuda
                    solver_precision=np.float32 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [9]:
x0

tensor([[-0.],
        [-0.],
        [-0.]], device='cuda:0')

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [10]:
x0 = torch.cat([x0, 
            torch.tensor([[.5], [.5], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[-0.0000, 0.5000],
        [-0.0000, 0.5000],
        [-0.0000, 0.5000]], device='cuda:0')

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [17]:
from mhar.walk import walk
X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=3**3, 
        device= None, 
        seed=None,
        verbosity=2
)

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  3   mI: 6   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


`walk` produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [18]:
X

tensor([[[-0.6127,  0.8089,  0.9789, -0.6988,  0.5368,  0.1097,  0.9752,
          -0.3432, -0.7830,  0.4428,  0.1547,  0.0728,  0.3892,  0.9601,
           0.1582,  0.9183, -0.3910, -0.2963, -0.0857, -0.1102,  0.8285,
          -0.5023, -0.5564,  0.2042,  0.4193,  0.5771, -0.4552, -0.0189,
           0.5479, -0.6841, -0.9025,  0.6026,  0.5671,  0.0486, -0.5358,
           0.0644,  0.8480,  0.0827, -0.1241,  0.6691,  0.1294,  0.1218,
           0.6437, -0.3485,  0.0271,  0.4424, -0.4029, -0.1587, -0.1941,
           0.1400,  0.4546, -0.0373, -0.8556, -0.9036, -0.5793,  0.9930,
           0.3530, -0.4537,  0.7579,  0.0417, -0.7985, -0.8154, -0.1402,
           0.8243, -0.2491,  0.6260,  0.7779,  0.2596,  0.4194,  0.1624,
          -0.1266,  0.3414, -0.3425, -0.1050, -0.5034,  0.6345, -0.8320,
           0.3718,  0.3144, -0.5869, -0.8809, -0.2634,  0.9745, -0.7064,
          -0.1577,  0.5086,  0.2419, -0.8145,  0.3439, -0.6100, -0.2184,
           0.7732,  0.8868,  0.0066,  0.1479, -0.38

In [19]:
X.shape

torch.Size([1, 3, 100])

### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a `Hypercube`.

In [13]:
from mhar.polytope_examples import Hypercube

# Create a polytope (Hypercube)
hypercube_sim = Hypercube(10,
                      dtype=torch.float32,
                      device='cuda'
                      )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



Define/Find inner points

In [14]:
x0_sim = ChebyshevCenter(polytope=hypercube_sim, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device='cuda',
                    solver_precision=np.float32)




Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [15]:
X_sim = walk(polytope=hypercube_sim,
        X0 = x0_sim,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X_sim

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  10   mI: 20   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[-3.1998e-01, -8.8300e-01, -6.0041e-01, -3.5785e-01,  9.4090e-03,
          -9.1916e-02, -7.6317e-01, -5.3222e-01, -8.9703e-01, -8.7063e-01,
          -1.1481e-02, -7.8231e-01,  8.1336e-01, -2.9168e-01,  9.4914e-01,
           5.9874e-01, -4.3935e-03,  2.1628e-01,  7.6173e-01,  2.6937e-01,
           2.4105e-01, -9.7515e-01,  1.5412e-01, -2.5267e-01,  1.0652e-02,
          -4.0451e-01,  9.0939e-01,  5.8938e-01,  4.9095e-01,  9.2517e-01,
          -6.8194e-01,  4.6277e-01,  9.4680e-01,  1.8939e-01, -9.4760e-01,
           1.5749e-01,  4.0214e-02, -8.7604e-01,  9.6388e-01, -1.2118e-01,
           8.3994e-01, -3.0281e-01, -9.9271e-01, -4.1503e-01,  6.7738e-01,
          -7.7913e-01,  1.5857e-01, -1.4566e-01,  2.9453e-01,  9.8420e-01,
           4.7988e-02,  9.3964e-01, -1.8877e-01, -4.8837e-01,  8.7884e-02,
          -6.5161e-01, -9.0205e-01, -4.3728e-01,  6.5158e-01,  9.5481e-01,
          -4.3082e-01,  2.6779e-01, -9.9665e-01,  2.7433e-01,  2.3229e-01,
          -8.9123e-01, -7

In [16]:
X_sim.shape

torch.Size([1, 10, 100])